# Build Network Graph

In [193]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import warnings
import random
warnings.filterwarnings('ignore', category=UserWarning)

np.random.seed(99) # Set random seed for reproducibility

df = pd.read_csv('../data/bilat_mig.csv')# Load the migration data
# Rebuild the directed graph
G = nx.DiGraph()
for index, row in df.iterrows():
    if row['da_pb_closed'] > 0:
        G.add_edge(row['orig'], row['dest'], weight=row['da_pb_closed'])
print("Libraries loaded and graph rebuilt successfully!")
print(f"Graph: {G.number_of_nodes()} countries, {G.number_of_edges()} migration flows")

Libraries loaded and graph rebuilt successfully!
Graph: 10 countries, 88 migration flows


## Test Traversing Graph

In [194]:
print(G.nodes)

# Set node parameters
G.nodes["USA"]["GDP"] = 1000

# Test retrieving data
print(G.nodes["USA"]["GDP"])

# Error if not set yet. Parameters are node-by-node
try:
    print(G.nodes["IND"]["GDP"])
except KeyError:
    print("Parameter not found")


# Test retrieving random
print("Random Node:",random.choice(list(G.nodes)))

['IND', 'PHL', 'POL', 'GBR', 'DEU', 'MEX', 'CAN', 'USA', 'AUS', 'SYR']
1000
Parameter not found
Random Node: SYR


# Modelling
## Agent Setup
Start with a simple agent that can traverse a networkX network.\
Uses template from wk 7 Lab:

Display gives the option to turn off print statements later

In [195]:
class Agent:
    """
    Summary of main attributes:
    id              - The "name" of this agent
    birth_place     - The source of this agent
    living_place    - The current living place
    job             - Fill later
    age             - Fill later
    wanderlust      - threshold to move per step
    """
    # A list of all locations lived in. If it doesn't move, a duplicate is added
    locations = []

    def __init__(self, id, birth_node):
        """Creates a new agent at the given location."""
        self.birth_place = birth_node
        self.living_place = birth_node
        self.job = "Builder"
        self.age = 0
        self.wanderlust = 0.3
        self.id = id



    def check_move(self, world, display=False):
        """Check if this agent wants to move. Update with more advanced cost function later"""
        if(random.random() < self.wanderlust):
            if(display): print("Checking", self.id, "... I live in ", self.living_place, "and I hate it")
            return True
        else:
            if(display): print("Checking", self.id, "... I live in ", self.living_place, "and I'm happy here")
            return False



    def step(self, world, display=False):
        self.age += 1

        if(self.check_move(world, display=display)):
            # Keep generating until we get a country that's not the same
            new_country = self.living_place
            while new_country == self.living_place:
                new_country = world.get_rand_country()

            if(display): print("    I'm moving to", new_country)
        else:
            # Unchanged, but add to locations array
            new_country = self.living_place
        
        # Add the new location to the array
        self.living_place = new_country
        self.locations.append(new_country)

## World Setup
Create a world environment to run and control enviro params

In [196]:
class World:
    """
    Summary of main attributes:
    houses? wealth? overpopulation?
    """

    agents = []

    def __init__(self, world_graph):
        self.world_graph = world_graph

    def get_rand_country(self):
        random_item = random.choice(list(self.world_graph.nodes))
        return random_item

    def add_agent(self):
        new_id = len(self.agents) + 1
        rand_birth_place = self.get_rand_country()
        new_agent = Agent(new_id, rand_birth_place)

        print("Adding new agent born in ", rand_birth_place)
        self.agents.append(new_agent)

    def get_agents(self):
        return self.agents
    
    def step(self):
        for agent in self.agents:
            agent.step(self, display=True) # Give the whole object so it can decision make

In [197]:
# Configure new world
random.seed(99) # Seed for this specific set of operations

world = World(G)

# Add 3 agents
world.add_agent()
world.add_agent()
world.add_agent()

for step in range(5):
    print("----- Generation", step, "-----")
    world.step()

    # # ask the agents again
    # for agent in world.get_agents():
    #     print("Hi, I'm", agent.id,"and I live in", agent.living_place)
    
    print("------------------------")

Adding new agent born in  CAN
Adding new agent born in  CAN
Adding new agent born in  GBR
----- Generation 0 -----
Checking 1 ... I live in  CAN and I'm happy here
Checking 2 ... I live in  CAN and I hate it
    I'm moving to POL
Checking 3 ... I live in  GBR and I'm happy here
------------------------
----- Generation 1 -----
Checking 1 ... I live in  CAN and I hate it
    I'm moving to AUS
Checking 2 ... I live in  POL and I'm happy here
Checking 3 ... I live in  GBR and I'm happy here
------------------------
----- Generation 2 -----
Checking 1 ... I live in  AUS and I'm happy here
Checking 2 ... I live in  POL and I'm happy here
Checking 3 ... I live in  GBR and I'm happy here
------------------------
----- Generation 3 -----
Checking 1 ... I live in  AUS and I'm happy here
Checking 2 ... I live in  POL and I hate it
    I'm moving to MEX
Checking 3 ... I live in  GBR and I'm happy here
------------------------
----- Generation 4 -----
Checking 1 ... I live in  AUS and I'm happy he